In [3]:
# Fast and Efficient Dataset Preparation for Parquet
import pandas as pd
import numpy as np
import random
from tqdm.auto import tqdm
import multiprocessing as mp
from functools import partial

Loading the required columns from the parquet File with address details

In [ ]:

# Method 1: Use columns directly during read to minimize memory usage
columns_to_load = ['OID_', 'State', 'Zip_Code', 'StreetAddress', 'CityStateZip', 'FullAddress', 'FormattedFullAddress', 'Latitude', 'Longitude']
df = pd.read_parquet('data/address_with_instructions.parquet', columns=columns_to_load)

# Normalize 'State' once to avoid repeat operations
df['State'] = df['State'].str.strip().str.lower()


Method for creating the dataset

In [ ]:
# --- Batchwise Processing ---
def process_ma_batches(df, total_count=5000, batch_size=500):
    ma_df = df[df['State'] == 'ma'].sample(n=total_count, random_state=42)
    ma_df = ma_df.sort_values('Zip_Code').reset_index(drop=True)

     # Convert 'Latitude' and 'Longitude' to numeric before processing
    ma_df['Latitude'] = pd.to_numeric(ma_df['Latitude'], errors='coerce')
    ma_df['Longitude'] = pd.to_numeric(ma_df['Longitude'], errors='coerce')

    total_batches = (len(ma_df) + batch_size - 1) // batch_size
    all_triplets = []

    for i in range(0, len(ma_df), batch_size):
        batch_df = ma_df.iloc[i:i+batch_size]
        print(f"Processing MA batch {i // batch_size + 1} of {total_batches}...")

        triplets = generate_triplets_parallel(
            batch_df,
            id_col='OID_',
            lat_col='Latitude',
            lon_col='Longitude',
            address_col='FormattedFullAddress',
            pos_thres=300,
            neg_thres=1000,
            max_triplets=None
        )

        all_triplets.extend(triplets)

    return all_triplets

# --- Triplet Generation Optimized ---
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def process_anchor(anchor_row, data, id_col, lat_col, lon_col, address_col, pos_thres, neg_thres):
    anchor_id = anchor_row[id_col]
    anchor_lat = anchor_row[lat_col]
    anchor_lon = anchor_row[lon_col]
    anchor_text = anchor_row[address_col]
    if pd.isna(anchor_lat) or pd.isna(anchor_lon):
        return []
    pos, neg = [], []

    for _, row in data.iterrows():
        if row[id_col] == anchor_id or pd.isna(row[lat_col]) or pd.isna(row[lon_col]):
            continue
        dist = haversine_distance(anchor_lat, anchor_lon, row[lat_col], row[lon_col])
        if dist <= pos_thres:
            pos.append(row[address_col])
        elif dist >= neg_thres:
            neg.append(row[address_col])

    if pos and neg:
        return [[anchor_text, random.choice(pos), random.choice(neg)]]
    return []

# Multiprocessing method

def generate_triplets_parallel(df, id_col, lat_col, lon_col, address_col, pos_thres, neg_thres, max_triplets=None):
    data = df[[id_col, lat_col, lon_col, address_col]]
    data_list = [row for _, row in data.iterrows()]

    pool = mp.Pool(mp.cpu_count())
    func = partial(process_anchor, data=data, id_col=id_col, lat_col=lat_col, lon_col=lon_col, address_col=address_col, pos_thres=pos_thres, neg_thres=neg_thres)

    triplets = []
    for result in tqdm(pool.imap_unordered(func, data_list), total=len(data_list)):
        if result:
            triplets.extend(result)
            if max_triplets and len(triplets) >= max_triplets:
                break

    pool.close()
    pool.join()
    return triplets[:max_triplets] if max_triplets else triplets


triplets = process_ma_batches(df, total_count=30000, batch_size=500)

triplets_df = pd.DataFrame(triplets, columns=['anchor', 'positive', 'negative'])
triplets_df.to_csv('data/retiever_triplets_dataset.csv', index=False)


Processing MA batch 1 of 60...


  0%|          | 0/500 [00:00<?, ?it/s]

Processing MA batch 2 of 60...


  0%|          | 0/500 [00:00<?, ?it/s]

Processing MA batch 3 of 60...


...

  0%|          | 0/500 [00:00<?, ?it/s]

Processing MA batch 59 of 60...


  0%|          | 0/500 [00:00<?, ?it/s]

Processing MA batch 60 of 60...


  0%|          | 0/500 [00:00<?, ?it/s]